In [1]:
import email
import csv
import re
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.precision = 18

In [22]:
import os
print(os.path.exists("trec06p-cs280/trec06p-cs280/labels")) 

False


### Understanding the Dataset 

Before we analyze and and perform analyzing strategies, we must first understand our dataset. 

The trec06p (Text Retrieval Conference) dataset is benchmark dataset to test and compar the performance of various machine learning and natural language processing approaches in determining between spam and legitimate (ham) emails. 

The dataset is made of large collection of emails pre-labeled as either spam or ham. This clear binary classification makes it ideal for supervised learning tasks. The emails also include features such as headers, body content and even metadata, which we can use to extract informative features for classification. 

# Data Preprocessing 

(add details here)

In [14]:


# =====================================================
# 1. Read the Labels and Split into Training and Test
# =====================================================

# Read the labels file using pandas
df_labels = pd.read_csv('trec06p-cs280\\labels', sep=' ', names=['class', 'file'])

# Use your snippet to define the training set:
split_index = df_labels.loc[df_labels['file'] == '../data/071/000'].index[0]
df_train_labels = df_labels.iloc[:split_index, :].reset_index(drop=True)
df_test_labels = df_labels.iloc[split_index:, :].reset_index(drop=True)

print("Training labels count:", len(df_train_labels))
print("Test labels count:", len(df_test_labels))


# =====================================================
# 2. Helper Function to Convert Relative Paths to Full Paths
# =====================================================

def get_full_path(relative_file, base_path="trec06p-cs280"):
    """
    Convert the relative file path given in the labels file to an absolute path.
    The labels use paths like '../data/xyz/filename', so we remove the '../'
    and join with the base_path.
    """
    # Remove '../' from the beginning of the string
    if relative_file.startswith('..'):
        relative_file = relative_file[3:]
    # Construct and return the full path
    return os.path.join(base_path, relative_file)


# =====================================================
# 3. Load Emails from the Given File Paths
# =====================================================

def load_emails(data_df, base_path="trec06p-cs280"):
    """
    Load email text and labels from the DataFrame.
    
    Parameters:
      - data_df: DataFrame containing 'class' and 'file' columns.
      - base_path: Base directory for the dataset files.
    
    Returns:
      - texts: List of the email text contents.
      - labels: List of corresponding labels.
    """
    texts = []
    labels = []
    for idx, row in data_df.iterrows():
        full_path = get_full_path(row['file'], base_path)
        try:
            # Using 'latin-1' encoding often works with older email datasets
            with open(full_path, 'r', encoding='latin-1') as file:
                content = file.read()
            texts.append(content)
            labels.append(row['class'])
        except Exception as e:
            print(f"Error reading {full_path}: {e}")
    return texts, labels

# Load emails for training and testing
train_emails, train_y = load_emails(df_train_labels)
test_emails, test_y = load_emails(df_test_labels)

print("Loaded training emails:", len(train_emails))
print("Loaded test emails:", len(test_emails))


# =====================================================
# 4. Build and Evaluate a Naive Bayes Classifier
# =====================================================

# Create a pipeline that vectorizes the text and trains a Naive Bayes classifier
pipeline = make_pipeline(
    CountVectorizer(stop_words='english'),
    MultinomialNB()
)

# Fit the model on the training data
pipeline.fit(train_emails, train_y)

# Predict labels on the test data
predictions = pipeline.predict(test_emails)

# Evaluate the performance using accuracy
accuracy = accuracy_score(test_y, predictions)
print("Test Accuracy: {:.4f}".format(accuracy))


FileNotFoundError: [Errno 2] No such file or directory: 'trec06p-cs280\\labels'